In [ ]:
import requests
import pandas as pd
import time
import random

API_KEY = "d0fetb1r01qsv9eh4490d0fetb1r01qsv9eh449g"  # Replace with your actual API key

tickers = [
    # "ADYEY",    # Adyen NV
    # "AAGIY",    # AIA Group Ltd
    # "EADSY",    # Airbus SE
    # "BABA",    # Alibaba Group Holding Ltd
    # "GOOGL",    # Alphabet Inc
    # "AMZN",    # Amazon.com Inc
    # "AAPL",    # Apple Inc
    # "ASML",    # ASML Holding NV
    # "AUR",    # Aurora Innovation Inc
    # "BNPQY",    # BNP Paribas SA
    # "BNTGY",    # Brenntag SE
    # "AVGO",    # Broadcom Inc
    # "BYDDY",    # BYD Co Ltd
    # "CODYY",    # Compagnie de Saint Gobain SA
    # "DHLGY",    # Deutsche Post AG
    # "FUTU",    # Futu Holdings Ltd
    # "GELYY",    # Geely Automobile Holdings Ltd
    # "HSAI",    # Hesai Group
    # "JD",    # JD.Com Inc
    # "AIQUY",    # L'Air Liquide Societe Anonyme pour l'Etude et l'Exploitation des Procedes Georges Claude SA
    # "LI",    # Li Auto Inc
    # "MPNGY",    # Meituan
    # "MSFT",    # Microsoft Corp
    # "MBLY",    # Mobileye Global Inc
    # "NIO",    # NIO Inc
    # "NVDA",    # NVIDIA Corp
    # "PDD",    # PDD Holdings Inc
    # "PNGAY",    # Ping An Insurance Group Co of China Ltd
    # "PONY",    # Pony AI Inc
    # "RNLSY",    # Renault SA
    # "SNY",    # Sanofi SA
    # "SAP",    # SAP SE
    # "SOAGY",    # Sartorius AG
    # "SIMO",    # Silicon Motion Technology Corp
    # "TSM",    # Taiwan Semiconductor Manufacturing Co Ltd
    # "TCEHY",    # Tencent Holdings Ltd
    # "TCOM",    # Trip.com Group Ltd
    # "WRD",    # WeRide Inc
    # "XPEV",    # Xpeng Inc
]

rows = []

def fetch_data(ticker):
    base_url = "https://finnhub.io/api/v1"

    row = {
        "Ticker": ticker,
        "Company": "N/A",
        "PE Ratio": "N/A",
        "Profit Margin": "N/A",
        "Mean Rating": "N/A",
        "Recommendation": "N/A",
        "No. of Analysts": "N/A",
        "Market Cap": "N/A",
        "Country": "N/A",
        "Industry": "N/A"
    }

    try:
        # Company profile
        profile = requests.get(f"{base_url}/stock/profile2?symbol={ticker}&token={API_KEY}").json()
        row["Company"] = profile.get("name", "N/A")
        row["Country"] = profile.get("country", "N/A")
        row["Industry"] = profile.get("finnhubIndustry", "N/A")
        mc = profile.get("marketCapitalization")
        if mc is not None:
            if mc >= 1_000_000:
                row["Market Cap"] = f"${mc / 1_000_000:.1f}T"
            else:
                row["Market Cap"] = f"${mc:.1f}B"

        # Financial metrics
        metrics = requests.get(f"{base_url}/stock/metric?symbol={ticker}&metric=all&token={API_KEY}").json().get("metric", {})
        pe = metrics.get("peNormalizedAnnual")
        if pe: row["PE Ratio"] = round(pe, 2)
        margin = metrics.get("netProfitMarginAnnual")
        if margin: row["Profit Margin"] = f"{round(margin, 2)}%"

        # Analyst recommendations
        recs = requests.get(f"{base_url}/stock/recommendation?symbol={ticker}&token={API_KEY}").json()
        if recs:
            latest = recs[0]
            counts = {
                "Strong Buy": latest.get("strongBuy", 0),
                "Buy": latest.get("buy", 0),
                "Hold": latest.get("hold", 0),
                "Sell": latest.get("sell", 0),
                "Strong Sell": latest.get("strongSell", 0)
            }
            total = sum(counts.values())
            row["No. of Analysts"] = total
            if total > 0:
                rating = (
                    counts["Strong Buy"] * 1 +
                    counts["Buy"] * 2 +
                    counts["Hold"] * 3 +
                    counts["Sell"] * 4 +
                    counts["Strong Sell"] * 5
                ) / total
                row["Mean Rating"] = round(rating, 2)
                row["Recommendation"] = (
                    "Buy" if rating <= 2 else
                    "Hold" if rating <= 3 else
                    "Sell"
                )

    except Exception as e:
        print(f"⚠️ Error with {ticker}: {e}")

    return row

# Loop through tickers
for ticker in tickers:
    print(f"Fetching {ticker}...")
    rows.append(fetch_data(ticker))
    time.sleep(random.uniform(1.2, 1.8))

# Create DataFrame
df = pd.DataFrame(rows)

# Export to CSV
df.to_csv("finnhub_stock_data.csv", index=False)

df


Fetching SIMO...
Fetching SNY...
Fetching BNPQY...
Fetching DHLGY...
Fetching RNLSY...
Fetching BNTGY...
Fetching SOAGY...


,Ticker,Company,PE Ratio,Profit Margin,Mean Rating,Recommendation,No. of Analysts,Market Cap,Country,Industry
0,SIMO,Silicon Motion Technology Corp,24.04,11.11%,2.06,Hold,16,$2146.1B,HK,Semiconductors
1,SNY,Sanofi SA,19.70,12.97%,2.00,Buy,29,$113157.9B,FR,Pharmaceuticals
2,BNPQY,BNP Paribas SA,7.46,14.57%,2.29,Hold,24,$87224.2B,FR,Banking
3,DHLGY,Deutsche Post AG,13.38,3.96%,2.32,Hold,25,$44568.5B,DE,Logistics & Transportation
4,RNLSY,Renault SA,18.51,1.34%,2.21,Hold,24,$13923.0B,FR,Automobiles
5,BNTGY,Brenntag SE,15.65,3.3%,2.38,Hold,26,$8391.7B,DE,Trading Companies & Distributors
6,SOAGY,Sartorius AG,163.83,2.48%,2.14,Hold,7,$13761.7B,DE,Life Sciences Tools & Services
